In [1]:
%load_ext autoreload
%autoreload 2
from practicalnlp import settings
from practicalnlp.data import Reader
from practicalnlp.training import fit
import torch.nn as nn
import torch


r = Reader((settings.TRAIN_DATA, settings.VALIDATION_DATA, settings.TEST_DATA))
train = r.load(settings.TRAIN_DATA)
valid = r.load(settings.VALIDATION_DATA)
test = r.load(settings.TEST_DATA)

# Training *LSTM* with random embeddings

The below code executes the classification using random embeddings. The `nn.Embedding` from PyTorch creates a `look-up` table from a word in the vocabulary to its respective embedding. If the programmer do not provide any pre-trained embeddings, the embeddings are initialized with random values and are trained using the `emb.weights` param, therefore, being a trainable parameter of the model.

In [46]:
from practicalnlp.models import LSTMClassifier

embed_dim = 300
embeddings = nn.Embedding(len(r.vocab), embed_dim)
model  = LSTMClassifier(embeddings, len(r.labels), embed_dim, 100, hidden_units=[100])

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters") 


model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(learnable_params, lr=1.0)

%time fit(model, r.labels, optimizer, loss, 10, 50, train, valid, test)

Model has 5342502 parameters
EPOCH 1
Training Results
{'acc': 0.6428320837826952, 'precision': 0.6500416471974484, 'recall': 0.7571641543813152, 'f1': 0.6995255897334995}
Validation Results
{'acc': 0.7442660550458715, 'precision': 0.7461024498886414, 'recall': 0.7545045045045045, 'f1': 0.7502799552071668}
New best model 0.74
EPOCH 2
Training Results
{'acc': 0.7472486064370265, 'precision': 0.7568413703011329, 'recall': 0.7951678932298445, 'f1': 0.7755313992937755}
Validation Results
{'acc': 0.786697247706422, 'precision': 0.76875, 'recall': 0.831081081081081, 'f1': 0.7987012987012988}
New best model 0.79
EPOCH 3
Training Results
{'acc': 0.7753537506009537, 'precision': 0.7865240739890761, 'recall': 0.810998840483684, 'f1': 0.7985739750445633}
Validation Results
{'acc': 0.8004587155963303, 'precision': 0.8229665071770335, 'recall': 0.7747747747747747, 'f1': 0.7981438515081206}
New best model 0.80
EPOCH 4
Training Results
{'acc': 0.7911279739088629, 'precision': 0.8019094179503736, 'reca

0.8094453596924767

# Training a *CNN* with random embeddings

The below code we train a convolutional neural network

In [2]:
from practicalnlp.models import ConvClassifier

embed_dim = 300
embeddings = nn.Embedding(len(r.vocab), embed_dim)
model  = ConvClassifier(embeddings, len(r.labels), embed_dim)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters")


model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(learnable_params, lr=1.0)

%time fit(model, r.labels, optimizer, loss, 10, 50, train, valid, test)

Model has 5442302 parameters
EPOCH 1
Training Results
{'acc': 0.5897142708644638, 'precision': 0.6157095509390638, 'recall': 0.6725904541044512, 'f1': 0.6428943023229513}
Validation Results
{'acc': 0.6697247706422018, 'precision': 0.6282894736842105, 'recall': 0.8603603603603603, 'f1': 0.7262357414448668}
New best model 0.67
EPOCH 2
Training Results
{'acc': 0.6350489208820052, 'precision': 0.6504203107752399, 'recall': 0.725052651506188, 'f1': 0.6857117280427898}
Validation Results
{'acc': 0.698394495412844, 'precision': 0.8389513108614233, 'recall': 0.5045045045045045, 'f1': 0.630098452883263}
New best model 0.70
EPOCH 3
Training Results
{'acc': 0.6644144436792662, 'precision': 0.6761351455644643, 'recall': 0.7463262263659812, 'f1': 0.7094989033237726}
Validation Results
{'acc': 0.6846330275229358, 'precision': 0.7478005865102639, 'recall': 0.5743243243243243, 'f1': 0.6496815286624203}
EPOCH 4
Training Results
{'acc': 0.6846584633775549, 'precision': 0.6941925734024179, 'recall': 0.76

0.741900054914882

# Training the same CNN with pre-trained embeddings

In [10]:
from practicalnlp.models import ConvClassifier
from practicalnlp import settings
from practicalnlp.data import EmbeddingsReader

embeddings, embed_dim = EmbeddingsReader.from_binary(settings.PRETRAINED_EMBEDDINGS_FILE, r.vocab)
model  = ConvClassifier(embeddings, len(r.labels), embed_dim)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters")


model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(learnable_params, lr=1.0)

%time fit(model, r.labels, optimizer, loss, 10, 50, train, valid, test)

Model has 5442302 parameters
EPOCH 1
Training Results
{'acc': 0.8338898922831044, 'precision': 0.8425174890185233, 'recall': 0.857829101493173, 'f1': 0.8501043547592806}
Validation Results
{'acc': 0.8004587155963303, 'precision': 0.7428057553956835, 'recall': 0.9301801801801802, 'f1': 0.8260000000000002}
New best model 0.80
EPOCH 2
Training Results
{'acc': 0.8801730746741856, 'precision': 0.8863705470961102, 'recall': 0.8967320570765991, 'f1': 0.8915211970074813}
Validation Results
{'acc': 0.8394495412844036, 'precision': 0.9021164021164021, 'recall': 0.7680180180180181, 'f1': 0.829683698296837}
New best model 0.84
EPOCH 3
Training Results
{'acc': 0.8960512467353595, 'precision': 0.9023370522607164, 'recall': 0.9090844553822854, 'f1': 0.9056981870476459}
Validation Results
{'acc': 0.8543577981651376, 'precision': 0.8610478359908884, 'recall': 0.8513513513513513, 'f1': 0.8561721404303511}
New best model 0.85
EPOCH 4
Training Results
{'acc': 0.9058224295422357, 'precision': 0.91138110121

0.8687534321801208